## Use Transformer for time-series forecasting  

The following code is deomo to apply Transformer encoder module to predict daily temperature. The `PyTorch` provides build-in `TransformerEncoderLayer` and `TransformerDncoderLayer`, so we needn't write it by ourselves. If you are interested in how to build an encoder and decoder from scratch. Please kindly read [pytorch_from_scratch.ipynb](./pytorch_from_scratch.ipynb)

Reference:  
[1] https://github.com/oliverguhr/transformer-time-series-prediction/tree/master

In [1]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot
import os

In [2]:
torch.manual_seed(0)
np.random.seed(0)

# This concept is also called teacher forceing. 
# The flag decides if the loss will be calculted over all 
# or just the predicted values.
calculate_loss_over_all_values = False

# S is the source sequence length
# T is the target sequence length
# N is the batch size
# E is the feature number

#src = torch.rand((10, 32, 512)) # (S,N,E) 
#tgt = torch.rand((20, 32, 512)) # (T,N,E)
#out = transformer_model(src, tgt)
#
#print(out)

input_window = 100
output_window = 5
batch_size = 10 # batch size
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]
       

class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [4]:
# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = np.append(input_data[i:i+tw][:-output_window] , output_window * [0])
        train_label = input_data[i:i+tw]
        #train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

def get_data():
    t        = np.arange(0, 400, 0.1)
    amplitude   = np.sin(t) + np.sin(t*0.05) +np.sin(t*0.12) *np.random.normal(-0.2, 0.2, len(t))
    
    # from pandas import read_csv
    # series = read_csv('./data/daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    # amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1) # (4000,)
    
    
    sampels = 2800
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment.. 
    train_sequence = create_inout_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window] #todo: fix hack?

    #test_data = torch.FloatTensor(test_data).view(-1) 
    test_data = create_inout_sequences(test_data,input_window)
    test_data = test_data[:-output_window] #todo: fix hack?

    return train_sequence.to(device), test_data.to(device)

def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target


def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i, batch_size)
        # data [100, 10, 1]
        # targets [100, 10, 1]
        optimizer.zero_grad()
        output = model(data) # [100, 10, 1]

        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
    
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            # look like the model returns static values for the output window
            output = eval_model(data)    
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) #todo: check this. -> looks good to me
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy()
    len(test_result)

    if not os.path.exists("./graph"):
        os.makedirs("./graph")

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i


def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    

    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png'%steps)
    pyplot.close()
        
# entweder ist hier ein fehler im loss oder in der train methode, aber die ergebnisse sind unterschiedlich 
# auch zu denen der predict_future
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            if calculate_loss_over_all_values:
                total_loss += len(data[0])* criterion(output, targets).cpu().item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()            
    return total_loss / len(data_source)

In [5]:
train_data, val_data = get_data()
model = TransAm().to(device)

criterion = nn.MSELoss()
lr = 0.005 
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)
    
    if(epoch % 10 == 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)
        
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step() 

<ipython-input-4-2947d93edf5f>:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.FloatTensor(inout_seq)
d:\softwears\Anaconda\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |    53/  269 batches | lr 0.005000 | 29.48 ms | loss 5.53086 | ppl   252.36
| epoch   1 |   106/  269 batches | lr 0.005000 |  3.40 ms | loss 0.17483 | ppl     1.19
| epoch   1 |   159/  269 batches | lr 0.005000 |  3.45 ms | loss 0.13412 | ppl     1.14
| epoch   1 |   212/  269 batches | lr 0.005000 |  3.40 ms | loss 0.21703 | ppl     1.24
| epoch   1 |   265/  269 batches | lr 0.005000 |  3.36 ms | loss 0.09104 | ppl     1.10
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  2.40s | valid loss 0.74906 | valid ppl     2.12
-----------------------------------------------------------------------------------------
| epoch   2 |    53/  269 batches | lr 0.004802 |  3.49 ms | loss 0.11046 | ppl     1.12
| epoch   2 |   106/  269 batches | lr 0.004802 |  3.44 ms | loss 0.06811 | ppl     1.07
| epoch   2 |   159/  269 batches | lr 0.004802 |  3.42 ms | loss 0.10746 | ppl     1.11
| epoch   2 |   212/  269 batche

d:\softwears\Anaconda\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch  11 |    53/  269 batches | lr 0.004004 |  3.49 ms | loss 0.02862 | ppl     1.03
| epoch  11 |   106/  269 batches | lr 0.004004 |  3.42 ms | loss 0.01439 | ppl     1.01
| epoch  11 |   159/  269 batches | lr 0.004004 |  3.51 ms | loss 0.01274 | ppl     1.01
| epoch  11 |   212/  269 batches | lr 0.004004 |  3.49 ms | loss 0.01204 | ppl     1.01
| epoch  11 |   265/  269 batches | lr 0.004004 |  3.46 ms | loss 0.01389 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  11 | time:  0.97s | valid loss 0.34629 | valid ppl     1.41
-----------------------------------------------------------------------------------------
| epoch  12 |    53/  269 batches | lr 0.003924 |  3.64 ms | loss 0.03075 | ppl     1.03
| epoch  12 |   106/  269 batches | lr 0.003924 |  3.51 ms | loss 0.01323 | ppl     1.01
| epoch  12 |   159/  269 batches | lr 0.003924 |  3.51 ms | loss 0.01275 | ppl     1.01
| epoch  12 |   212/  269 batche

d:\softwears\Anaconda\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

| epoch  71 |    53/  269 batches | lr 0.001191 |  3.51 ms | loss 0.01529 | ppl     1.02
| epoch  71 |   106/  269 batches | lr 0.001191 |  3.46 ms | loss 0.00988 | ppl     1.01
| epoch  71 |   159/  269 batches | lr 0.001191 |  3.49 ms | loss 0.01267 | ppl     1.01
| epoch  71 |   212/  269 batches | lr 0.001191 |  3.40 ms | loss 0.01511 | ppl     1.02
| epoch  71 |   265/  269 batches | lr 0.001191 |  3.42 ms | loss 0.01489 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  71 | time:  0.97s | valid loss 0.01527 | valid ppl     1.02
-----------------------------------------------------------------------------------------
| epoch  72 |    53/  269 batches | lr 0.001167 |  3.53 ms | loss 0.01416 | ppl     1.01
| epoch  72 |   106/  269 batches | lr 0.001167 |  3.55 ms | loss 0.01057 | ppl     1.01
| epoch  72 |   159/  269 batches | lr 0.001167 |  3.64 ms | loss 0.01171 | ppl     1.01
| epoch  72 |   212/  269 batche

d:\softwears\Anaconda\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch  81 |   106/  269 batches | lr 0.000973 |  3.49 ms | loss 0.00892 | ppl     1.01
| epoch  81 |   159/  269 batches | lr 0.000973 |  3.61 ms | loss 0.01063 | ppl     1.01
| epoch  81 |   212/  269 batches | lr 0.000973 |  3.47 ms | loss 0.01248 | ppl     1.01
| epoch  81 |   265/  269 batches | lr 0.000973 |  3.45 ms | loss 0.01194 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  81 | time:  0.98s | valid loss 0.01777 | valid ppl     1.02
-----------------------------------------------------------------------------------------
| epoch  82 |    53/  269 batches | lr 0.000954 |  3.49 ms | loss 0.01147 | ppl     1.01
| epoch  82 |   106/  269 batches | lr 0.000954 |  3.44 ms | loss 0.00902 | ppl     1.01
| epoch  82 |   159/  269 batches | lr 0.000954 |  3.38 ms | loss 0.01098 | ppl     1.01
| epoch  82 |   212/  269 batches | lr 0.000954 |  3.40 ms | loss 0.01277 | ppl     1.01
| epoch  82 |   265/  269 batche

d:\softwears\Anaconda\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch  91 |   106/  269 batches | lr 0.000795 |  3.59 ms | loss 0.00824 | ppl     1.01
| epoch  91 |   159/  269 batches | lr 0.000795 |  3.97 ms | loss 0.00939 | ppl     1.01
| epoch  91 |   212/  269 batches | lr 0.000795 |  3.77 ms | loss 0.00976 | ppl     1.01
| epoch  91 |   265/  269 batches | lr 0.000795 |  3.66 ms | loss 0.01004 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  91 | time:  1.05s | valid loss 0.01061 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  92 |    53/  269 batches | lr 0.000779 |  3.67 ms | loss 0.00996 | ppl     1.01
| epoch  92 |   106/  269 batches | lr 0.000779 |  3.55 ms | loss 0.00801 | ppl     1.01
| epoch  92 |   159/  269 batches | lr 0.000779 |  3.63 ms | loss 0.00925 | ppl     1.01
| epoch  92 |   212/  269 batches | lr 0.000779 |  3.69 ms | loss 0.00997 | ppl     1.01
| epoch  92 |   265/  269 batche